# Toronto Clustering

#### Importing needed libraries:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


#### Get the file into a Dataframe:

In [30]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

In [31]:
print(df.shape)
df.head(10)

(290, 3)


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


#### Drop cells with a borough that is Not assigned:

In [32]:
df = df[df[1]!='Not assigned']
df.head(15)

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge


#### Now the shape of the Dataframe is:

In [33]:
df.shape

(213, 3)

#### So 77 raws deleted.

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. (see line 9):

In [34]:
df.iloc[7,2]=df.iloc[7,1] #(index=7)
df.head(10)

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge


#### Reseting index:

In [35]:
df.reset_index(inplace=True)
df.head()

,index,0,1,2
0,0,Postcode,Borough,Neighbourhood
1,3,M3A,North York,Parkwoods
2,4,M4A,North York,Victoria Village
3,5,M5A,Downtown Toronto,Harbourfront
4,6,M5A,Downtown Toronto,Regent Park


In [36]:
df.drop('index',axis=1, inplace=True)
df.head(6)

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights


#### More than one neighborhood can exist in one postal code area:

In [37]:
df1 = df[df[0].duplicated()==True].reset_index() #duplicate dataframe
df1.head()

,index,0,1,2
0,4,M5A,Downtown Toronto,Regent Park
1,6,M6A,North York,Lawrence Manor
2,10,M1B,Scarborough,Malvern
3,13,M4B,East York,Parkview Hill
4,15,M5B,Downtown Toronto,Garden District


In [38]:
s=0
for i in df1.iloc[:,0].tolist():
    for j in range(i):
        if df.iloc[j,0]==df1.iloc[s,1]:
            df.iloc[j,2]=df.iloc[j,2]+','+df1.iloc[s,3]
    s=s+1

In [39]:
df

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Rouge,Malvern"


In [40]:
df.drop(df1['index'].tolist(),axis=0,inplace=True)

#### Get it in the final form:

In [41]:
df.rename(columns={0:'Postcode', 1:'Borough', 2:'Neighbourhood'},inplace=True) #rename columns
df.drop(index=0,inplace=True) #drop column raw
df.reset_index(inplace=True) #reset index
df.drop('index',axis=1,inplace=True) #drop index column
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [42]:
df.shape

(103, 3)